In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cofactor.model import LatticePredictor, FEATURES, OUTPUTS
from cofactor.calculator import CofactorCalculator as CC
from cofactor.calculator import Lattice

In [4]:
model = LatticePredictor.from_features()

In [8]:
tetragonal = pd.read_csv('data/tetragonal.csv')
tetragonal = tetragonal[tetragonal.Zr > 0]

monoclinic = pd.read_csv('data/monoclinic.csv')
monoclinic = monoclinic[monoclinic.Zr > 0]

In [21]:
tetragonal = tetragonal.rename(columns={'a': 'tetr_a', 'c': 'tetr_c'})
monoclinic = monoclinic.rename(columns={'a': 'mono_a', 'b': 'mono_b', 'c': 'mono_c', 'beta': 'mono_beta'})

In [49]:
df = pd.concat([tetragonal, monoclinic], sort=False, ignore_index=True)

In [50]:
out = 'tetr_a'
idx = df[FEATURES + [out]].dropna().index

In [54]:
model.fit_df(df)